In [ ]:
import os 
import yaml 
import pickle
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 


plt.rcParams['font.size'] = 14 
plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['lines.linewidth'] = .5

In [ ]:
from bindata_processing.base import bindata_query 
from bindata_processing import plots 

In [ ]:
with open('user_input.yaml', 'r') as f:
    inputs = yaml.safe_load(f)

query = bindata_query( inputs )
query.load_dict()

In [ ]:
plots.print_dict_hierarchy(query.main_dict )

In [ ]:
plots.plot_all_orb2( query , 'Orb2_diff' )

In [ ]:
date = query.dates_formated[0]
st1_df = query.main_dict[date]['Strike'][48300.0]
st2_df = query.main_dict[date]['Strike'][50000.0]
date 

In [ ]:
plots.plot_time_series( st1_df , 'Orb2_diff' , 48300.0 , query.main_dict[date]['Stamp'] , date , query.inputs  ) 
plots.plot_time_series( st2_df , 'Orb2_diff' , 50000.0 , query.main_dict[date]['Stamp'] , date , query.inputs ) 

In [ ]:
strikes_df = { 48300.0 : st1_df , 50000.0 : st2_df }

### KPSS 

In [ ]:
from statsmodels.tsa.stattools import kpss

for strike , df in strikes_df.items() : 
    temp = {}

    stat1, p_value1, lags1, crit1 = kpss(df['Orb2_diff'][1:], regression='c') 

    temp['stat1'] = stat1 
    temp['p_value1']  = p_value1 
    temp['crit1'] = crit1

    

    print(f'KPSS Results of Rejection: strike = {strike}')
    for k , v in temp.items() : 
        print( f'{k}: {v}' )
    print({ k : stat1 > v for k, v in temp['crit1'].items() } )
    print('-'*20)

In [ ]:
from statsmodels.tsa.stattools import adfuller

for strike , df in strikes_df.items() : 
    temp = {}

    stat1, p_value1, lags1, _ ,  crit1  , _ = adfuller(df['Orb2_diff'][1:]  ) 

    temp['stat1'] = stat1 
    temp['p_value1']  = p_value1 
    temp['crit1'] = crit1



    print(f'ADF Results of Rejection: strike: {strike}' )
    for k , v in temp.items() : 
        print( f'{k}: {v}' )
    print( f'p-value < 0.05: {p_value1 < 0.05 }' )
    print( { k : stat1 < v for k, v in temp['crit1'].items() } )
    print('-'*20)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA


def grid_search_ARIMA( data , max_p = 3  , max_q = 3 , method = 'AIC') : 

    best = np.inf
    best_order = None

    for p in range(1,max_p + 1):
        for q in range(1,max_q + 1):

            try:
                model = ARIMA(data, order=(p, 0, q))  # d=0 means no differencing here
                result = model.fit()
                if method == 'AIC' : 
                    if result.aic < best:
                        best = result.aic
                        best_order = (p, q)
                if method == 'BIC' : 
                    if result.bic < best:
                        best = result.bic
                        best_order = (p, q)
            except Exception as e:
                # Sometimes model won't converge or fail; skip those
                continue

    return best_order , best 

In [ ]:
for strike , df in strikes_df.items() : 
    aic_order , aic_val = grid_search_ARIMA(df['Orb2_diff'][1:])
    bic_order , bic_val = grid_search_ARIMA(df['Orb2_diff'][1:] , method = 'BIC' )

    temp = {
        'aic_order' : aic_order , 'aic_val' : aic_val , 
        'bic_order' : bic_order , 'bic_val' : bic_val 
    }

    print(f'Strike = {strike}')
    for k , v in temp.items() : 
        print( k , v )
    print('-'*20)

In [ ]:
from hurst import compute_Hc 

for strike , df in strikes_df.items() : 
    H , *_  =  compute_Hc(df['Orb2_diff'][1:])
    print(f'Hurst Exponent for {strike} = {H}')

### FOR Strike = 50000.0

In [ ]:
model = ARIMA( st2_df['Orb2_diff'] , order = ( 2 , 0 , 2 ) )
fit2 = model.fit()
print( fit2.summary() )

fit2.plot_diagnostics(figsize=(10,8))
plt.show()

In [ ]:
model = ARIMA( st2_df['Orb2_diff'] , order = ( 2 , 1 , 2 ) )
fit2 = model.fit()
print( fit2.summary() )

fit2.plot_diagnostics(figsize=(10,8))
plt.show()

### FOR Strike = 48300.0 

In [ ]:
model = ARIMA( st1_df['Orb2_diff'] , order = ( 2 , 0 , 3 ) )
fit1 = model.fit()
print( fit1.summary() )

fit1.plot_diagnostics(figsize=(10,8))
plt.show()

In [ ]:
model = ARIMA( st1_df['Orb2_diff'] , order = ( 2 , 1 , 3 ) )
fit1 = model.fit()
print( fit1.summary() )

fit1.plot_diagnostics(figsize=(10,8))
plt.show()

In [ ]:
model = ARIMA( st1_df['Orb2_diff'] , order = ( 1 , 0 , 3 ) )
fit1 = model.fit()
print( fit1.summary() )

fit1.plot_diagnostics(figsize=(10,8))
plt.show()